In [0]:
from pyspark.sql.functions import *
from calendar import Calendar

In [0]:
def mnameToNo(dt):
    mname = dt[3:6].upper()
    calendar = {"JAN": "01", "FEB": "02", "MAR": "03", "APR": "04",
                "MAY": "05", "JUN": "06", "JUL": "07", "AUG": "08", "SEP": "09", "OCT": "10",
                "NOV": "11", "DEC": "12"}
    return dt.upper().replace(mname, calendar[mname])

In [0]:
udf_mname_to_no = udf(mnameToNo)

In [0]:
def group_df(input_df):
    output_df = input_df.drop("_c15").\
    withColumn('rts', to_timestamp(udf_mname_to_no("TIMESTAMP"), "dd-MM-yyyy"))
    grouped_df = output_df.groupBy('rts', 'SYMBOL','INSTRUMENT','EXPIRY_DT','OPTION_TYP','YEAR', 'MONTH', 'DAY')\
    .agg(sum('contracts').alias('Contracts'),
    sum('VAL_INLAKH').alias('VALUE'),
    sum('OPEN_INT').alias('OPEN_INT'))
    return grouped_df

In [0]:
def add_pcr_to_df(input_df):
    return input_df\
    .filter("INSTRUMENT != 'FUTSTK'")\
    .groupBy('rts', 'SYMBOL', 'EXPIRY_DT', 'YEAR', 'MONTH', 'DAY')\
    .pivot('OPTION_TYP', ['XX','PE','CE'])\
    .agg(sum('Contracts').alias('contracts'))\
    .withColumn('pcr', col('PE')/col('CE'))

In [0]:
def combine_grouped_and_pcr_dfs(grouped_df, pcr_df):
    return grouped_df.alias('fgi').join(pcr_df.alias('pcr'),['rts', 'SYMBOL', 'EXPIRY_DT'])\
.select('rts','SYMBOL', 'EXPIRY_DT', 'INSTRUMENT', 'OPTION_TYP', 'CONTRACTS', 'VALUE', 'OPEN_INT', 'PCR','fgi.YEAR', 'fgi.MONTH', 'fgi.DAY')

In [0]:
def write_processed_output(processed_df,output_path,output_mode,table_name):
    processed_df.write\
    .partitionBy('YEAR','MONTH', 'DAY')\
    .mode(output_mode)\
    .option('path', output_path)\
    .format('parquet')\
    .saveAsTable(table_name)

In [0]:
def generate_target_location(filename):
    year = filename[7:11]
    months_dictionary = {'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6,
                         'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12}
    month = months_dictionary[filename[4:7].upper()]
    day = filename[2:4]
    if day[0] == '0':
        day = day.replace('0','')
    return f'year={year}/month={month}/day={day}'

In [0]:
def validate_date(yr,month,dy):
    if not dy.isdigit():
        return False
    day_to_check = int(dy)
    if (day_to_check in [x.day for x in Calendar().itermonthdates(int(yr), int(month))]):
            return True
    else:
        return False
    

In [0]:
if validate_date(2020,12,'29'):
    print('yep')
else:
    print('no')

yep